In [5]:
import pandas as pd
import numpy as np

In [6]:
df=pd.read_csv("energydata.csv")

In [7]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
x1 = df[["T2"]]
y1 = df[["T6"]]
x_train, x_test, y_train, y_test = train_test_split(x1, y1, test_size=0.3, random_state=42)

lr = LinearRegression()

lr.fit(x_train, y_train)

y = lr.predict(x_test)

round(r2_score(y_test, y),2)

0.64

In [9]:
df = df.drop(columns = ['date', 'lights'])


In [16]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error


scaler = MinMaxScaler()
nm_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

feature = nm_df.drop(columns=['Appliances'])
target = nm_df['Appliances']

x_train, x_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, random_state=42)

lr.fit(x_train, y_train)

predicted_values = lr.predict(x_test)

round(mean_absolute_error(y_test, predicted_values),2)

0.05

In [15]:
round(np.sum(np.square(y_test - predicted_values)),2)

45.35

In [14]:
from sklearn.metrics import  mean_squared_error
round(np.sqrt(mean_squared_error(y_test, predicted_values)),3)

0.088

In [28]:
from sklearn.metrics import r2_score
round(r2_score(y_test, predicted_values),2)

-1.67

In [34]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=0.4)

ridge_reg.fit(x_train, y_train)

#obtain predictions
y_hat = ridge_reg.predict(x_test)

round(np.sqrt(mean_squared_error(y_test, y_hat)),3)

3.63

In [35]:
from sklearn.linear_model import Lasso

las_reg = Lasso(alpha=0.001)

las_reg.fit(x_train, y_train)

y_hat = las_reg.predict(x_test)

def get_weights_df(model, feat, col_name):
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name] = weights_df[col_name]
  return weights_df

lasso_weights_df = get_weights_df(las_reg, x_train, 'Lasso_weight')
(lasso_weights_df['Lasso_weight']==0).value_counts()



False    1
Name: Lasso_weight, dtype: int64

In [36]:
round(np.sqrt(mean_squared_error(y_test, y_hat)),3)

3.63